# Setup

In [1]:
%matplotlib inline

import numpy as np
import scipy.signal as sig
import scipy.stats as stat
import matplotlib.pyplot as plt
import seaborn as sns
import os
import h5py
import pandas as pd

from pandas import DataFrame,Series,read_table

In [2]:
savePlots = False    # whether or not to save plots
saveData = False # whether or not to save csv files

saveAsPath = '../formatted_scores/'


In [3]:
birdPaths =    ['../data_copies/01_PreprocessedData/01_BudgieFemale_green1/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/02_BudgieMale_yellow1/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/03_BudgieFemale_white1/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/04_BudgieMale_yellow2/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/05_BudgieFemale_green2/00_Baseline_night/']

arfFilePaths =  ['EEG 2 scored/',
                 'EEG 3 scored/',
                 'EEG 3 scored/',
                 'EEG 4 scored/',
                 'EEG 4 scored/']

### load BEST EEG channels - as determined during manual scoring ####
channelsToLoadEEG_best = [['5 LEEGf-LEEGp', '6 LEEGm-LEEGp'],
                     ['5 LEEGf-LEEGm'], #, '4 LEEGf-Fgr'] this channel seems to have too much artifact
                     ['6LEEGm-LEEGp', '9REEGm-REEGp'],
                     ['9REEGf-REEGp', '6LEEGm-LEEGf'],
                     ['4LEEGf-LEEGp','7REEGf-REEGp']]


### load ALL of EEG channels ####
channelsToLoadEEG = [['4 LEEGf-Fgr', '5 LEEGf-LEEGp', '6 LEEGm-LEEGp', '7 LEEGp-Fgr', '8 REEGp-Fgr','9 REEGp-LEEGp'],
                     ['4 LEEGf-Fgr','5 LEEGf-LEEGm', '6 LEEGm-LEEGp', '7 REEGf-Fgr', '8 REEGm-Fgr', '9 REEGf-REEGm'],
                     ['4LEEGf-LEEGp', '5LEEGf-LEEGm', '6LEEGm-LEEGp', '7REEGf-REEGp', '8REEGf-REEGm', '9REEGm-REEGp'],
                     ['4LEEGf-LEEGp', '5LEEGm-LEEGp', '6LEEGm-LEEGf', '7REEGf-Fgr', '8REEGf-REEGm','9REEGf-REEGp',],
                     ['4LEEGf-LEEGp', '5LEEGf-LEEGm', '6LEEGm-LEEGp', '7REEGf-REEGp', '8REEGf-REEGm', '9REEGm-REEGp']]


channelsToLoadEOG = [['1 LEOG-Fgr', '2 REOG-Fgr'],
                     ['2 LEOG-Fgr', '3 REOG-Fgr'],
                     ['2LEOG-Fgr', '3REOG-Fgr'],
                     ['2LEOG-Fgr', '3REOG-Fgr'],
                     ['2LEOG-Fgr', '3REOG-Fgr']]

birds_LL = [1,2,3]
nBirds_LL = len(birds_LL)

birdPaths_LL = ['../data_copies/01_PreprocessedData/02_BudgieMale_yellow1/01_Constant_light/',
                '../data_copies/01_PreprocessedData/03_BudgieFemale_white1/01_Constant_light/',
                '../data_copies/01_PreprocessedData/04_BudgieMale_yellow2/01_Constant_light/',]

arfFilePaths_LL =  ['EEG 2 preprocessed/',
                    'EEG 2 preprocessed/',
                    'EEG 2 preprocessed/']

lightsOffSec = np.array([7947, 9675, 9861 + 8*3600, 9873, 13467])  # lights off times in seconds from beginning of file
lightsOnSec = np.array([46449, 48168, 48375+ 8*3600, 48381, 52005]) # Bird 3 gets 8 hours added b/c file starts at 8:00 instead of 16:00

epochLength = 3
sr = 200
scalingFactor = (2**15)*0.195       # scaling/conversion factor from amplitude to uV (when recording arf from jrecord)

stages = ['w','d','u','i','s','r'] # wake, drowsy, unihem sleep, intermediate sleep, SWS, REM
stagesSleep =    ['u','i','s','r']

stagesVideo = ['m','q','d','s','u'] # moving wake, quiet wake, drowsy, sleep, unclear

## Path to scores formatted as CSVs
formatted_scores_path = '../formatted_scores/'


Calculate general variables

In [4]:
lightsOffEp = lightsOffSec / epochLength
lightsOnEp = lightsOnSec / epochLength

nBirds = len(birdPaths)
birds = np.arange(0, nBirds)

epochLengthPts = epochLength*sr

nStages = len(stagesSleep)

# Read in manual & video scores: DIRECTLY FROM ARF FILES

## LL

In [5]:
birdPaths_LL

['../data_copies/01_PreprocessedData/02_BudgieMale_yellow1/01_Constant_light/',
 '../data_copies/01_PreprocessedData/03_BudgieFemale_white1/01_Constant_light/',
 '../data_copies/01_PreprocessedData/04_BudgieMale_yellow2/01_Constant_light/']

In [19]:
# LL

AllScores = {}

for b in [1,2]:

    video_scores_path = birdPaths_LL[b] + 'Video/'

    # init
    all_scores_array = np.array([], dtype='str')
    all_video_scores_array = np.array([], dtype='str')
    all_datetime_array = np.array([], dtype='datetime64')
    all_video_datetime_array = np.array([], dtype='datetime64')

    # go through each arf file in order
    for file in list(np.sort(os.listdir(video_scores_path))):
        if file.endswith('epochs.csv'):
            # open csv file
            raw_scores = pd.read_csv(video_scores_path + file)

            # Get datetime from file name
            date = file.split('_')[2]
            time = file.split('_')[3]
            hours = time.split('-')[0]
            minutes = time.split('-')[1]
            datetime_start = np.datetime64(date + 'T' + hours + ':' + minutes + ':06')    # assume 6-s delay in starting recording

            ### SCORES ###

            # Get scores

            arf_video_scores = raw_scores['Label']   # Video
            all_video_scores_array = np.append(all_video_scores_array, arf_video_scores)

            ### TIMES ###

            # Get start times of each epoch in datetime format

            arf_video_starttimes_s = raw_scores['Start (s)']     # Video
            arf_video_datetimes = datetime_start + arf_video_starttimes_s.astype('timedelta64[s]')
            all_video_datetime_array = np.append(all_video_datetime_array, arf_video_datetimes)

        # Save in dateframe, in dict under bird number
        #BirdScores  = DataFrame(all_scores_array, index=all_datetime_array, columns=['Label'])
        videoScores = DataFrame(all_video_scores_array, index=all_video_datetime_array, columns=['Video Label'])
        BirdScores = videoScores   #.join(BirdScores)

        data_name = 'Bird ' + str(b+2) + '_LL'
        AllScores[data_name] = BirdScores

In [20]:
data_names = list(AllScores.keys())
data_names

['Bird 3_LL', 'Bird 4_LL']

In [21]:
AllScores

{'Bird 3_LL':                     Video Label
 2014-12-25 16:00:06           d
 2014-12-25 16:00:09           s
 2014-12-25 16:00:12           d
 2014-12-25 16:00:15           d
 2014-12-25 16:00:18           q
 ...                         ...
 2014-12-26 15:59:45           d
 2014-12-26 15:59:48           s
 2014-12-26 15:59:51           s
 2014-12-26 15:59:54           m
 2014-12-26 15:59:57           q
 
 [28687 rows x 1 columns], 'Bird 4_LL':                     Video Label
 2015-02-17 17:10:06           d
 2015-02-17 17:10:09           d
 2015-02-17 17:10:12           d
 2015-02-17 17:10:15           d
 2015-02-17 17:10:18           d
 ...                         ...
 2015-02-18 17:10:51           s
 2015-02-18 17:10:54           s
 2015-02-18 17:10:57           s
 2015-02-18 17:11:00           s
 2015-02-18 17:11:03           s
 
 [28707 rows x 1 columns]}

#### Convert datetimes to s, ep, Zeitgeber time

In [22]:
# LD #

lightsOffDatetime = np.array([], dtype='datetime64')
lightsOnDatetime = np.array([], dtype='datetime64')

for b_name in data_names:
    b_num = int(b_name[5]) - 1

    Scores = AllScores[b_name]
    startDatetime = Scores.index.values[0]

    # Calc lights off & on using datetime formats
    lightsOffTimedelta = lightsOffSec[b_num].astype('timedelta64[s]')
    lightsOffDatetime = np.append(lightsOffDatetime, startDatetime + lightsOffTimedelta)
    lightsOnTimedelta = lightsOnSec[b_num].astype('timedelta64[s]')
    lightsOnDatetime = np.append(lightsOnDatetime, startDatetime + lightsOnTimedelta)

    # Convert to seconds & hours from start of recording
    timedelta_ns = Scores.index.values - startDatetime
    timedelta_s  = timedelta_ns.astype('timedelta64[s]')

    time_s = timedelta_s.astype('int')
    time_hr = time_s/3600

    AllScores[b_name]['Time (s)'] = time_s
    AllScores[b_name]['Time (h)'] = time_hr

    # Convert to Zeitgeber time
    time_Zeit_s = time_s - lightsOnSec[b_num]
    time_Zeit_hr = time_Zeit_s/3600
    time_Zeir_ep = (time_Zeit_s/3).astype('int')

    AllScores[b_name]['Zeitgeber time (s)'] = time_Zeit_s
    AllScores[b_name]['Zeitgeber time (h)'] = time_Zeit_hr
    AllScores[b_name]['Zeitgeber time (ep)'] = time_Zeir_ep

    # Convert to epoch number
    ind_epoch = (time_s/3).astype('int')

    AllScores[b_name]['Epoch #'] = ind_epoch

In [23]:
AllScores

{'Bird 3_LL':                     Video Label  Time (s)   Time (h)  Zeitgeber time (s)  \
 2014-12-25 16:00:06           d         0   0.000000              -77175   
 2014-12-25 16:00:09           s         3   0.000833              -77172   
 2014-12-25 16:00:12           d         6   0.001667              -77169   
 2014-12-25 16:00:15           d         9   0.002500              -77166   
 2014-12-25 16:00:18           q        12   0.003333              -77163   
 ...                         ...       ...        ...                 ...   
 2014-12-26 15:59:45           d     86379  23.994167                9204   
 2014-12-26 15:59:48           s     86382  23.995000                9207   
 2014-12-26 15:59:51           s     86385  23.995833                9210   
 2014-12-26 15:59:54           m     86388  23.996667                9213   
 2014-12-26 15:59:57           q     86391  23.997500                9216   
 
                      Zeitgeber time (h)  Zeitgeber time (ep)

#### Calculate lights off in Zeitgeber time (s and hrs)
Lights on is 0 

In [24]:
lightsOffZeit_s = lightsOffSec - lightsOnSec
lightsOffZeit_hr = lightsOffZeit_s / 3600

# Save to csv

In [25]:
for b in [1,2]: 
    b_name = 'Bird ' + str(b+2)
    data_name = b_name + '_LL'
    scores = AllScores[data_name]
    if saveData:
        scores.to_csv(saveAsPath + 'All_scores_' + b_name + 'LL.csv')